## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [5]:
print('Instrument:', instrument)

Instrument: EUR_USD


### The Configs for Run:

In [6]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [7]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [8]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [9]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [10]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [11]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
       response=second_response 
else:
    response=first_response['candles']

In [12]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [13]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data.shape

(9666, 12)

In [15]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-11-29,22:00:00,22,333,6,Sunday,2015-11-29T22:00:00.000000000Z,2657,1.05902,1.05945,1.05706,1.05794
1,2015-11-30,02:00:00,2,334,0,Monday,2015-11-30T02:00:00.000000000Z,1062,1.05798,1.05918,1.05770,1.05853
2,2015-11-30,06:00:00,6,334,0,Monday,2015-11-30T06:00:00.000000000Z,4372,1.05856,1.05951,1.05636,1.05810
3,2015-11-30,10:00:00,10,334,0,Monday,2015-11-30T10:00:00.000000000Z,4910,1.05813,1.05849,1.05646,1.05748
4,2015-11-30,14:00:00,14,334,0,Monday,2015-11-30T14:00:00.000000000Z,6511,1.05752,1.05854,1.05578,1.05598


In [16]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9661,2022-02-15,14:00:00,14,46,1,Tuesday,2022-02-15T14:00:00.000000000Z,42569,1.13558,1.13684,1.13202,1.13588
9662,2022-02-15,18:00:00,18,46,1,Tuesday,2022-02-15T18:00:00.000000000Z,11357,1.13588,1.13644,1.13546,1.13586
9663,2022-02-15,22:00:00,22,46,1,Tuesday,2022-02-15T22:00:00.000000000Z,6075,1.13580,1.13622,1.13487,1.13521
9664,2022-02-16,02:00:00,2,47,2,Wednesday,2022-02-16T02:00:00.000000000Z,7719,1.13521,1.13556,1.13450,1.13540
9665,2022-02-16,06:00:00,6,47,2,Wednesday,2022-02-16T06:00:00.000000000Z,176,1.13539,1.13550,1.13534,1.13545


## Simple Moving Average (SMA)

In [17]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [18]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [19]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [20]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9661,2022-02-15,14:00:00,14,46,1,Tuesday,2022-02-15T14:00:00.000000000Z,42569,1.13558,1.13684,1.13202,1.13588,1.1338,1.1329,1.1362,0.00208,0.00298,-0.00032
9662,2022-02-15,18:00:00,18,46,1,Tuesday,2022-02-15T18:00:00.000000000Z,11357,1.13588,1.13644,1.13546,1.13586,1.1347,1.1329,1.1360,0.00116,0.00296,-0.00014
9663,2022-02-15,22:00:00,22,46,1,Tuesday,2022-02-15T22:00:00.000000000Z,6075,1.13580,1.13622,1.13487,1.13521,1.1354,1.1333,1.1354,-0.00019,0.00191,-0.00019
9664,2022-02-16,02:00:00,2,47,2,Wednesday,2022-02-16T02:00:00.000000000Z,7719,1.13521,1.13556,1.13450,1.13540,1.1356,1.1337,1.1351,-0.00020,0.00170,0.00030
9665,2022-02-16,06:00:00,6,47,2,Wednesday,2022-02-16T06:00:00.000000000Z,176,1.13539,1.13550,1.13534,1.13545,1.1356,1.1341,1.1348,-0.00015,0.00135,0.00065


## Price Range

In [21]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [22]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [23]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
9642,2022-02-15,14:00:00,14,46,1,Tuesday,2022-02-15T14:00:00.000000000Z,42569,1.13558,1.13684,...,-0.00030,0.00482,0.00096,-0.00386,1,-0.00030,0.00096,0.00356,-0.00098,-0.00288
9643,2022-02-15,18:00:00,18,46,1,Tuesday,2022-02-15T18:00:00.000000000Z,11357,1.13588,1.13644,...,0.00002,0.00098,0.00058,-0.00040,0,0.00002,0.00058,0.00042,-0.00030,-0.00098
9644,2022-02-15,22:00:00,22,46,1,Tuesday,2022-02-15T22:00:00.000000000Z,6075,1.13580,1.13622,...,0.00059,0.00135,0.00101,-0.00034,0,0.00059,0.00101,0.00093,0.00002,-0.00030
9645,2022-02-16,02:00:00,2,47,2,Wednesday,2022-02-16T02:00:00.000000000Z,7719,1.13521,1.13556,...,-0.00019,0.00106,0.00016,-0.00090,1,-0.00019,0.00016,0.00071,0.00059,0.00002
9646,2022-02-16,06:00:00,6,47,2,Wednesday,2022-02-16T06:00:00.000000000Z,176,1.13539,1.13550,...,-0.00006,0.00016,0.00005,-0.00011,1,-0.00006,0.00005,0.00005,-0.00019,0.00059


In [24]:
data.shape

(9647, 30)